In [42]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd


In [43]:
import pdfkit
import PyPDF2

In [44]:
from PyPDF2 import PdfFileReader,PdfFileWriter

In [45]:
from os import path

In [15]:
import os

In [46]:
import time

In [47]:
save_fold = r'e:\pandastest'

In [48]:
out_file = r'e:\pandastest\1.pdf'

In [49]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.101 Safari/537.36"}

In [50]:
url = "https://www.jianshu.com/p/fb6719a18cea"

In [51]:
info_last = {'title':' 附录B 更多关于IPython的内容（完）','url':url}

In [52]:
html_template = '''
<!DOCTYPE html>
<html lang = 'en'>
<head>
    <meta charset = 'UTF-8'>
</head>
<body>
{content}
</body>
</html>
'''

In [53]:
options = {  
        'page-size': 'Letter',  
        'margin-top': '0.75in',  
        'margin-right': '0.75in',  
        'margin-bottom': '0.75in',  
        'margin-left': '0.75in',  
        'encoding': "UTF-8",  
        'custom-header': [  
            ('Accept-Encoding', 'gzip')  
        ],  
        'cookie': [  
            ('cookie-name1', 'cookie-value1'),  
            ('cookie-name2', 'cookie-value2'),  
        ],  
        'outline-depth': 10,  
    }



In [54]:
def generate_url(url_html):
    
    '''
    从给定的网页中产生需要解析的网址
    '''
    charter_info = [] #存储章节信息，包括标题及网址
    bsj = BeautifulSoup(url_html.text,'lxml')
    url_list = bsj.select('div.show-content-free a') #css选择，也可以作为函数参数
    for url in url_list[:15]: #根据情况逐个处理链接，填充charter_info列表
        info = {}
        info['title'] =str(url.string)
        info['url'] = url['href']
        charter_info.append(info)
    charter_info.append(info_last) #因为本页也含义数据，将传入的链接也放入列表
    return charter_info

In [55]:
def get_html(url):
    
    """
    爬取html文档
    """
    try:
        html = requests.get(url,headers = headers)
    except:
        print('fail:',url)
    return html

In [56]:
def get_content(html):
    
    """
    从爬取的网页中提取需要的内容，注意取消了不需要的连接
    """
    bsj = BeautifulSoup(html.text,'lxml')
    bsj = bsj.select('div.show-content-free')
    bsj = bsj[0] #转换列表
    bsj_a = bsj.select('a[href*=www.jianshu.com]') #以下几行剔除不需要的内容
    bsj_a[0].parent.decompose() 
    bsj_a[-1].parent.decompose()
    html_content = html_template.format(content = bsj) #生成完整的HTML文档，为生成pdf做准备
    return html_content

In [57]:
def save_pdf(html,filename):
    """
    保存pdf文档
    """
    pdfkit.from_string(html,filename,options)

In [58]:
def merge_pdf(infnList, outfn):
    
    """
    合并PDF文档
    参数infnList 需要合并的pdf文件列表；outfn合并后的文件名称及路径
    """
    pages_total = 0
    pdf_output = PdfFileWriter() #创建输出对象
    for infn in infnList: 
        pdf_input = PdfFileReader(open(infn['filename'], 'rb'),strict = False)
        # 获取 pdf 共用多少页
        page_count = pdf_input.getNumPages()
        
        print(page_count)
        for i in range(page_count):#逐页插入输出的文档
            pdf_output.addPage(pdf_input.getPage(i))
        pdf_output.addBookmark(infn['bookmark'],pagenum = pages_total) #生成标签
        pages_total += page_count #计算下一个标签的位置
    pdf_output.write(open(outfn, 'wb')) #输出


In [61]:
def main():
    infnList = []
    html = requests.get(url,headers = headers) #爬取初始文档
    charter_info = generate_url(html)
    for charter in charter_info:
        html = get_html(charter['url'])
        time.sleep(2)
        content = get_content(html)
        filename = path.join(save_fold,charter['title'] + '.pdf')
        save_pdf(content,filename)
    for chart in charter_info: #生成需要合并的文档信息，包括标签即各章节名称、文档位置及名称
        infn = {'bookmark':chart['title'],'filename':path.join(save_fold,chart['title']+
                                                              '.pdf')}
        infnList.append(infn)
    merge_pdf(infnList,out_file)

In [62]:
main()

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                       